In [16]:
#Importação das bibliotecas necessárias

import os
import pandas as pd
import numpy as np
import urllib.parse
import qrcode
import re
print ("Biblioteca importada com sucesso!")

Biblioteca importada com sucesso!


In [17]:
#Definir constantes para caminhos e URLs

URL_BASE_QR = "https://datacanuto.github.io/galeria-digital-artes"

# Obter diretório base do repositório (onde está o notebook)
DIRETORIO_BASE = r'D:\Documentos\REPOSITÓRIO-LOCAL\GaleriaDigital'
# Definir caminhos relativos ao repositório
IMG_PATH = os.path.join(DIRETORIO_BASE, "galeria-digital-artes", "assets", "img")
DIRETORIO_SAIDA = os.path.join(DIRETORIO_BASE, "galeria-digital-artes", "links_obras")
QR_EXPORT_DIR = os.path.join(DIRETORIO_BASE, "galeria-digital-artes", "qr_codes_export")
DADOS = os.path.join(DIRETORIO_BASE, "dados_obras.csv")
DADOS_TRATADOS = os.path.join(DIRETORIO_BASE, "obras_com_links.csv")

# Criar diretório para exportação de QR codes
os.makedirs(QR_EXPORT_DIR, exist_ok=True)
print(f"Diretório base: {DIRETORIO_BASE}")
print(f"Diretório de imagens: {IMG_PATH}")
print(f"Diretório de exportação de QR codes: {QR_EXPORT_DIR}")

Diretório base: D:\Documentos\REPOSITÓRIO-LOCAL\GaleriaDigital
Diretório de imagens: D:\Documentos\REPOSITÓRIO-LOCAL\GaleriaDigital\galeria-digital-artes\assets\img
Diretório de exportação de QR codes: D:\Documentos\REPOSITÓRIO-LOCAL\GaleriaDigital\galeria-digital-artes\qr_codes_export


In [18]:
#Criação do DataFrame a partir da dados fornecidos, e exibição dos primeiros registros

df = pd.read_csv(DADOS)
df = pd.DataFrame(df)
print("Dados carregados com sucesso!")
print("Total de registros:", len(df))
df.head(20)
#df.tail()

Dados carregados com sucesso!
Total de registros: 62


,item,telas,técnica,dimensão,ano,valor (r$),valor_pix,parcelamento,categoria
0,1,PORTAS E JANELAS 1,ACRÍLICA SOBRE TELA,100 X 60,2018,"6.000,00","5.700,00",5% À VISTA/ 6 VEZES,TELAS
1,2,PORTAS E JANELAS 2,ACRÍLICA SOBRE TELA,70 X 120,2021,"8.400,00","7.980,00",5% À VISTA/ 6 VEZES,TELAS
2,3,PORTAS E JANELAS 3,ACRÍLICA SOBRE TELA,53 X 73,2025,"3.860,00","3.667,00",5% À VISTA/ 6 VEZES,TELAS
3,4,PORTAS E JANELAS 4,ACRÍLICA SOBRE TELA,50 X 70,2024,"3.500,00","3.325,00",5% À VISTA/ 6 VEZES,TELAS
4,5,PORTAS E JANELAS 5,ACRÍLICA SOBRE TELA,70 X 100,2025,"7.000,00","6.650,00",5% À VISTA/ 6 VEZES,TELAS
5,6,CASA CURITIBA 1,ACRÍLICA SOBRE TELA,70 X 120,2019,"6.720,00","6.384,00",5% À VISTA/ 6 VEZES,TELAS
6,7,CASA CURITIBA 2,ACRÍLICA SOBRE TELA,70 X 120,2019,"6.720,00","6.384,00",5% À VISTA/ 6 VEZES,TELAS
7,8,FUSCA VERMELHO,ACRÍLICA SOBRE TELA,70 X 50,2006,"3.500,00","3.325,00",5% À VISTA/ 6 VEZES,TELAS
8,9,WALKING,ACRÍLICA SOBRE TELA,75 x 102,2024,"7.500,00","7.125,00",5% À VISTA/ 6 VEZES,TELAS
9,10,SUCATA,ACRÍLICA SOBRE TELA,50 X 70,2005,ACERVO PESSOAL,ACERVO PESSOAL,NaN,TELAS


In [19]:
#Exibir informações do DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   item          62 non-null     int64 
 1   telas         62 non-null     object
 2   técnica       62 non-null     object
 3   dimensão      62 non-null     object
 4   ano           62 non-null     int64 
 5   valor (r$)    62 non-null     object
 6   valor_pix     62 non-null     object
 7   parcelamento  60 non-null     object
 8   categoria     62 non-null     object
dtypes: int64(2), object(7)
memory usage: 4.5+ KB


In [20]:
#Normalizar os nomes das colunas do DataFrame
df.columns = df.columns.str.strip()
df.columns = df.columns.str.lower()

print(df.columns)

Index(['item', 'telas', 'técnica', 'dimensão', 'ano', 'valor (r$)',
       'valor_pix', 'parcelamento', 'categoria'],
      dtype='object')


In [21]:
# Listar imagens no diretório e criar lista com caminhos completos
img_list = os.listdir(IMG_PATH)
img_fullpath = [os.path.join(IMG_PATH, img) for img in img_list]

# Criar dicionário para mapear índice com imagem
img_dict = {i+1: img for i, img in enumerate(img_list)}
df['fotos'] = df['item'].map(img_dict)

In [22]:
#Exibir informações do DataFrame após adição da coluna de fotos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   item          62 non-null     int64 
 1   telas         62 non-null     object
 2   técnica       62 non-null     object
 3   dimensão      62 non-null     object
 4   ano           62 non-null     int64 
 5   valor (r$)    62 non-null     object
 6   valor_pix     62 non-null     object
 7   parcelamento  60 non-null     object
 8   categoria     62 non-null     object
 9   fotos         62 non-null     object
dtypes: int64(2), object(8)
memory usage: 5.0+ KB


In [23]:
# Criar coluna 'status' [disponível, acervo pessoal]
df['status'] = np.where(df['valor (r$)'] != 0, 'disponível', 'acervo pessoal')
print(df[['telas','status']])

                 telas      status
0   PORTAS E JANELAS 1  disponível
1   PORTAS E JANELAS 2  disponível
2   PORTAS E JANELAS 3  disponível
3   PORTAS E JANELAS 4  disponível
4   PORTAS E JANELAS 5  disponível
..                 ...         ...
57              BIKE 2  disponível
58           LAMBRETTA  disponível
59           CHEVROLET  disponível
60             KOMBI 3  disponível
61             GORDINI  disponível

[62 rows x 2 columns]


In [24]:
# Criar coluna 'op_parcelamento' extraindo número inteiro após a barra '/'
df['op_parcelamento'] = df['parcelamento'].apply(lambda x: re.search(r'/\s*(\d+)', str(x)).group(1) if pd.notna(x) and re.search(r'/\s*(\d+)', str(x)) else '0')
print(df[['telas','status','op_parcelamento']])

                 telas      status op_parcelamento
0   PORTAS E JANELAS 1  disponível               6
1   PORTAS E JANELAS 2  disponível               6
2   PORTAS E JANELAS 3  disponível               6
3   PORTAS E JANELAS 4  disponível               6
4   PORTAS E JANELAS 5  disponível               6
..                 ...         ...             ...
57              BIKE 2  disponível               3
58           LAMBRETTA  disponível               3
59           CHEVROLET  disponível               3
60             KOMBI 3  disponível               3
61             GORDINI  disponível               3

[62 rows x 3 columns]


In [25]:
df.head()

,item,telas,técnica,dimensão,ano,valor (r$),valor_pix,parcelamento,categoria,fotos,status,op_parcelamento
0,1,PORTAS E JANELAS 1,ACRÍLICA SOBRE TELA,100 X 60,2018,"6.000,00","5.700,00",5% À VISTA/ 6 VEZES,TELAS,REF 01...PORTAS E JANELAS 1.jpg,disponível,6
1,2,PORTAS E JANELAS 2,ACRÍLICA SOBRE TELA,70 X 120,2021,"8.400,00","7.980,00",5% À VISTA/ 6 VEZES,TELAS,REF 02...PORTAS E JANELAS 2.jpg,disponível,6
2,3,PORTAS E JANELAS 3,ACRÍLICA SOBRE TELA,53 X 73,2025,"3.860,00","3.667,00",5% À VISTA/ 6 VEZES,TELAS,REF 03...PORTAS E JANELAS 3.jpg,disponível,6
3,4,PORTAS E JANELAS 4,ACRÍLICA SOBRE TELA,50 X 70,2024,"3.500,00","3.325,00",5% À VISTA/ 6 VEZES,TELAS,REF 04 ...PORTAS E JANELAS 4.jpg,disponível,6
4,5,PORTAS E JANELAS 5,ACRÍLICA SOBRE TELA,70 X 100,2025,"7.000,00","6.650,00",5% À VISTA/ 6 VEZES,TELAS,REF 05 - PORTAS E JANELAS 5.jpg,disponível,6


In [26]:
# Carregar template HTML único
TEMPLATE_OBRA = os.path.join(DIRETORIO_BASE, "galeria-digital-artes", "assets", "template_obra.html")

with open(TEMPLATE_OBRA, 'r', encoding='utf-8') as f:
    template_obra = f.read()

In [27]:
# Funções auxiliares

def gerar_html_obra(row):
    """Gera o HTML completo da obra substituindo placeholders, exibe preço, info de parcelamento e link da obra"""
    titulo = row.get('telas', '')
    titulo_encoded = urllib.parse.quote(str(titulo))
    tecnica_dimensao_ano = f"{row.get('técnica', '')}, {row.get('dimensão', '')}, {row.get('ano','')}"
    preco = row.get('valor (r$)', 0)
    preco_str = f"R$ {preco}" if preco else "Sob consulta"
    parcelamento = row.get('parcelamento', '')
    parcelamento_str = f"{parcelamento} SEM JUROS" if parcelamento else ""
    link_obra = row.get('qr link', '')
    html = template_obra.replace('{{TITULO}}', titulo) \
                        .replace('{{FOTO}}', row.get('fotos', '')) \
                        .replace('{{TECNICA_DIMENSAO_ANO}}', tecnica_dimensao_ano) \
                        .replace('{{TITULO_ENCODED}}', titulo_encoded) \
                        .replace('{{URL_BASE}}', URL_BASE_QR) \
                        .replace('{{PRECO}}', preco_str) \
                        .replace('{{PARCELAMENTO}}', parcelamento_str) \
                        .replace('{{LINK_OBRA}}', link_obra)
    return html

def gerar_qrcode(link, caminho_salvar):
    """Gera e salva um QR code"""
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(link)
    qr.make(fit=True)
    imagem_qr = qr.make_image(fill_color="black", back_color="white")
    imagem_qr.save(caminho_salvar)


In [28]:
# Loop principal: Gerador de etiquetas
for i, row in df.iterrows():
    idx_val = int(row.get('item', i))
    pasta = f"item_{idx_val}"
    caminho_pasta = os.path.join(DIRETORIO_SAIDA, pasta)
    os.makedirs(caminho_pasta, exist_ok=True)
    
    link_final = f"{URL_BASE_QR}/links_obras/{pasta}/index.html"
    df.at[i, 'qr link'] = link_final
    
    # Gerar e salvar HTML
    html_content = gerar_html_obra(row)
    with open(os.path.join(caminho_pasta, "index.html"), "w", encoding="utf-8") as f:
        f.write(html_content)
    
    # Gerar QR codes
    gerar_qrcode(link_final, os.path.join(caminho_pasta, "qrcode.png"))
    
    qr_export_nome = f"qr_{idx_val}.png"
    gerar_qrcode(link_final, os.path.join(QR_EXPORT_DIR, qr_export_nome))
    df.at[i, 'qr_export_file'] = qr_export_nome

print("✅ Páginas HTML e QR Codes gerados com sucesso!")
print(f"📁 QR Codes exportados para: {QR_EXPORT_DIR}")
print(f"\n📊 Resumo:")


✅ Páginas HTML e QR Codes gerados com sucesso!
📁 QR Codes exportados para: D:\Documentos\REPOSITÓRIO-LOCAL\GaleriaDigital\galeria-digital-artes\qr_codes_export

📊 Resumo:


In [29]:
# Salvar CSV com links de pagamento no repositório
csv_output = os.path.join(DIRETORIO_BASE, "obras_com_links.csv")
df.to_csv(csv_output, index=False, encoding="utf-8")
print(f"CSV salvo em: {csv_output}")
df.head()

CSV salvo em: D:\Documentos\REPOSITÓRIO-LOCAL\GaleriaDigital\obras_com_links.csv


,item,telas,técnica,dimensão,ano,valor (r$),valor_pix,parcelamento,categoria,fotos,status,op_parcelamento,qr link,qr_export_file
0,1,PORTAS E JANELAS 1,ACRÍLICA SOBRE TELA,100 X 60,2018,"6.000,00","5.700,00",5% À VISTA/ 6 VEZES,TELAS,REF 01...PORTAS E JANELAS 1.jpg,disponível,6,https://datacanuto.github.io/galeria-digital-a...,qr_1.png
1,2,PORTAS E JANELAS 2,ACRÍLICA SOBRE TELA,70 X 120,2021,"8.400,00","7.980,00",5% À VISTA/ 6 VEZES,TELAS,REF 02...PORTAS E JANELAS 2.jpg,disponível,6,https://datacanuto.github.io/galeria-digital-a...,qr_2.png
2,3,PORTAS E JANELAS 3,ACRÍLICA SOBRE TELA,53 X 73,2025,"3.860,00","3.667,00",5% À VISTA/ 6 VEZES,TELAS,REF 03...PORTAS E JANELAS 3.jpg,disponível,6,https://datacanuto.github.io/galeria-digital-a...,qr_3.png
3,4,PORTAS E JANELAS 4,ACRÍLICA SOBRE TELA,50 X 70,2024,"3.500,00","3.325,00",5% À VISTA/ 6 VEZES,TELAS,REF 04 ...PORTAS E JANELAS 4.jpg,disponível,6,https://datacanuto.github.io/galeria-digital-a...,qr_4.png
4,5,PORTAS E JANELAS 5,ACRÍLICA SOBRE TELA,70 X 100,2025,"7.000,00","6.650,00",5% À VISTA/ 6 VEZES,TELAS,REF 05 - PORTAS E JANELAS 5.jpg,disponível,6,https://datacanuto.github.io/galeria-digital-a...,qr_5.png


In [30]:
'''# Gerar QR Code para o catálogo completo
catalogo_link = "https://datacanuto.github.io/galeria-digital-artes/catalog_mobile/index.html"

qr_catalogo = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)
qr_catalogo.add_data(catalogo_link)
qr_catalogo.make(fit=True)

imagem_qr_catalogo = qr_catalogo.make_image(fill_color="black", back_color="white")

# Salvar QR code do catálogo
qr_catalogo_path = os.path.join(QR_EXPORT_DIR, "qr_catalogo_completo.png")
imagem_qr_catalogo.save(qr_catalogo_path)

print(f"QR Code do catálogo gerado com sucesso!")
print(f"Salvo em: {qr_catalogo_path}")'''

'# Gerar QR Code para o catálogo completo\ncatalogo_link = "https://datacanuto.github.io/galeria-digital-artes/catalog_mobile/index.html"\n\nqr_catalogo = qrcode.QRCode(\n    version=1,\n    error_correction=qrcode.constants.ERROR_CORRECT_L,\n    box_size=10,\n    border=4,\n)\nqr_catalogo.add_data(catalogo_link)\nqr_catalogo.make(fit=True)\n\nimagem_qr_catalogo = qr_catalogo.make_image(fill_color="black", back_color="white")\n\n# Salvar QR code do catálogo\nqr_catalogo_path = os.path.join(QR_EXPORT_DIR, "qr_catalogo_completo.png")\nimagem_qr_catalogo.save(qr_catalogo_path)\n\nprint(f"QR Code do catálogo gerado com sucesso!")\nprint(f"Salvo em: {qr_catalogo_path}")'